### Training the Video Classification Model

### This script does - VGG16

In [2]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import pickle

import time


 




In [3]:
Drive = "C:"
## Define root folder
RootFolder = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles"




In [4]:
trainFrames  = pd.read_csv(RootFolder+"/trainFrames.csv")

trainFrames.head()

,FrameFilename,FullPathName,class
0,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [5]:
trainFrames.shape

(2072, 3)

In [6]:
# Get the unique values of 'class' column - How many distinct classes
model_class = trainFrames['class'].unique().tolist()
print(model_class)
model_output_size = len(model_class)
print(model_output_size)


['Basketball', 'SoccerPenalty']
2


In [7]:
trainFrames.shape[0]

2072

In [8]:
trainFrames['FullPathName'][0]

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting\\VDOFrames/MoreBasketBallVdo_frame0.jpg'

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import pafy

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import train_image 

In [10]:
train_image = []

# for loop to read and store frames
for i in tqdm(range(trainFrames.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    # three-dimensional array containing the red, green, and blue pixel intensities. 
    #Image has been reformated to be 224-by-224 pixels in size.
    img = image.load_img(trainFrames['FullPathName'][i], target_size=(224,224,3))
    #print(img.shape)
    # converting it to array
    img = image.img_to_array(img)
    
    # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|█████████████████████████████████████████████████████████████████████████████| 2072/2072 [00:18<00:00, 113.88it/s]


(2072, 224, 224, 3)

In [11]:
# make sure that the distribution of each class is similar in both training and validation sets. 
# We can use the stratify parameter

# separating the target
y = trainFrames['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [12]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

### Defining the architecture of the video classification model

### VGG16 on ImageNet

https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1


#### Keras provides a set of state-of-the-art deep learning models along with pre-trained weights on ImageNet. These pre-trained models can be used for image classification, feature extraction, and transfer learning

##### VGG16 refers to a VGG model with 16 weight layers, and VGG19 refers to a VGG model with 19 weight layers.  
#####  1.The input layer takes an image in the size of (224 x 224 x 3), and the output layer is a softmax prediction on classes.

#####  2.From the input layer to the last max pooling layer (labeled by 7 x 7 x 512) is regarded as the feature extraction part of the model, 

##### 3 while the rest of the network is regarded as the classification part of the model.

## VGG-16 pre-trained model will be used
### https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

### creating the base model of pre-trained VGG16 model


In [13]:

# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

### set include_top = False it overlooks the last 3 Fully Connected(FC) layers 


base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### vgg16 (without last 3 Fully Connected(FC)) is an impressively large neural network. It has 14,714,688 parameters. 
To see all paratemer use 
vgg19_full = VGG19(weights='imagenet')
vgg19_full.summary()

In [14]:

%%time
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape


Wall time: 19.8 s


(1657, 7, 7, 512)

In [15]:
%%time
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape


Wall time: 5.41 s


(415, 7, 7, 512)

In [18]:
print(X_train.shape)
print(X_test.shape)
 
    

(1657, 7, 7, 512)
(415, 7, 7, 512)


There are 415 images in the test set and the shape of these images has also changed to (7, 7, 512). We will use a fully connected network now to fine-tune the model. This fully connected network takes input in single dimension. So, we will reshape the images into a single dimension:

In [19]:
X_train = X_train.reshape(X_train.shape[0], 7*7*512)
X_test = X_test.reshape(X_test.shape[0], 7*7*512)

In [20]:
# normalize the pixel values, i.e., keep the pixel values between 0 and 1. This helps the model to converge faster.

In [21]:
X_train.max()

10.658496

In [22]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [23]:
# shape of images
X_train.shape

(1657, 25088)

### Keras Neural Network Sequential Model
##### More Info - https://faroit.com/keras-docs/1.0.1/getting-started/sequential-model-guide/

In [24]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(model_output_size, activation='softmax'))  # model_output_size is 2 - already defined



###### 1 x Dense layer of 1024 units

###### 1 x Dense layer of 512 units
###### 1 x Dense layer of 256 units
###### 1 x Dense layer of 128 units


#### RELU activation for first four dense layers of  units to stop forwarding negative values through the network.

#### 2 unit dense layer in the end with softmax activation as there are 2 classes (model_output_size) to predict from in the end which are Basketball and SoccerPenalty. 
#### The softmax layer will output the value between 0 and 1 based on the confidence of the model that which class the images belongs to.

## Dropout
####   Usually, dropout is placed on the fully connected layers only because they are the one with the greater number of parameters and thus they're likely to excessively co-adapting themselves causing overfitting. However, since it's a stochastic regularization technique, you can really place it everywhere
#####  Dropout here has been applied applied to hidden neurons in the body of network model.
#### a dropout rate of 50% is used 

In [25]:
# Printing the models summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

#### ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

### -----

### save_best_only: if save_best_only=True, it only saves when the model is considered the "best" and the latest best model according to the quantity monitored will not be overwritten

### The metric name to monitormode: one of {'auto', 'min', 'max'}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For val_acc, this should be max, for val_loss this should be min

#### mode: one of {'auto', 'min', 'max'}. If save_best_only=True, -- the decision to overwrite the current save file is made based on either the maximization or the minimization  of the monitored quantity. For val_acc, this should be max, for val_loss this should be min

In [26]:
###Training the video classification model

# defining a function to save the weights of best model
### from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')




#### using Adam optimiser 

In [27]:
# compiling the model - Adding loss, optimizer and metrics values to the model.
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])



In [28]:
%%time
# training the model
#model_training_history = model.fit( X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

# Adding Early Stopping Callback
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

model_training_history = model.fit(x = X_train, y = y_train, epochs = 50, verbose = 1, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks=[mcp_save])

Train on 1325 samples, validate on 332 samples
Epoch 1/50
1325/1325 [==============================] - ETA: 2:06 - loss: 0.7391 - accuracy: 0.0000e+ - ETA: 48s - loss: 0.6904 - accuracy: 0.3333     - ETA: 33s - loss: 0.8451 - accuracy: 0.350 - ETA: 26s - loss: 0.7433 - accuracy: 0.500 - ETA: 22s - loss: 0.8819 - accuracy: 0.444 - ETA: 20s - loss: 0.9819 - accuracy: 0.454 - ETA: 18s - loss: 0.9503 - accuracy: 0.480 - ETA: 17s - loss: 0.9110 - accuracy: 0.516 - ETA: 16s - loss: 0.8951 - accuracy: 0.544 - ETA: 15s - loss: 0.9171 - accuracy: 0.526 - ETA: 14s - loss: 0.9685 - accuracy: 0.511 - ETA: 14s - loss: 1.1097 - accuracy: 0.489 - ETA: 13s - loss: 1.1134 - accuracy: 0.500 - ETA: 13s - loss: 1.0683 - accuracy: 0.518 - ETA: 13s - loss: 1.1685 - accuracy: 0.500 - ETA: 12s - loss: 1.1321 - accuracy: 0.516 - ETA: 12s - loss: 1.1596 - accuracy: 0.515 - ETA: 12s - loss: 1.1639 - accuracy: 0.514 - ETA: 12s - loss: 1.1468 - accuracy: 0.527 - ETA: 11s - loss: 1.1464 - accuracy: 0.525 - ETA: 11s

1325/1325 [==============================] - ETA: 9s - loss: 0.0055 - accuracy: 1.00 - ETA: 10s - loss: 0.2889 - accuracy: 0.916 - ETA: 10s - loss: 0.1742 - accuracy: 0.950 - ETA: 10s - loss: 0.1246 - accuracy: 0.964 - ETA: 10s - loss: 0.1006 - accuracy: 0.972 - ETA: 11s - loss: 0.0848 - accuracy: 0.977 - ETA: 11s - loss: 0.0724 - accuracy: 0.980 - ETA: 11s - loss: 0.0755 - accuracy: 0.983 - ETA: 11s - loss: 0.0666 - accuracy: 0.985 - ETA: 11s - loss: 0.0617 - accuracy: 0.986 - ETA: 11s - loss: 0.0559 - accuracy: 0.988 - ETA: 11s - loss: 0.0513 - accuracy: 0.989 - ETA: 11s - loss: 0.0504 - accuracy: 0.990 - ETA: 11s - loss: 0.0768 - accuracy: 0.981 - ETA: 11s - loss: 0.0734 - accuracy: 0.982 - ETA: 11s - loss: 0.0788 - accuracy: 0.975 - ETA: 10s - loss: 0.0741 - accuracy: 0.977 - ETA: 10s - loss: 0.0701 - accuracy: 0.978 - ETA: 10s - loss: 0.0671 - accuracy: 0.979 - ETA: 10s - loss: 0.0791 - accuracy: 0.974 - ETA: 10s - loss: 0.0753 - accuracy: 0.975 - ETA: 10s - loss: 0.0723 - accurac

1325/1325 [==============================] - ETA: 17s - loss: 5.6624e-07 - accuracy: 1.000 - ETA: 14s - loss: 3.2075e-05 - accuracy: 1.000 - ETA: 19s - loss: 3.4963e-05 - accuracy: 1.000 - ETA: 16s - loss: 3.0475e-04 - accuracy: 1.000 - ETA: 15s - loss: 0.0016 - accuracy: 1.0000    - ETA: 14s - loss: 0.0116 - accuracy: 1.000 - ETA: 13s - loss: 0.0097 - accuracy: 1.000 - ETA: 13s - loss: 0.0088 - accuracy: 1.000 - ETA: 12s - loss: 0.0077 - accuracy: 1.000 - ETA: 12s - loss: 0.0069 - accuracy: 1.000 - ETA: 12s - loss: 0.0062 - accuracy: 1.000 - ETA: 11s - loss: 0.0057 - accuracy: 1.000 - ETA: 11s - loss: 0.0055 - accuracy: 1.000 - ETA: 11s - loss: 0.0051 - accuracy: 1.000 - ETA: 11s - loss: 0.0048 - accuracy: 1.000 - ETA: 11s - loss: 0.0045 - accuracy: 1.000 - ETA: 11s - loss: 0.0047 - accuracy: 1.000 - ETA: 10s - loss: 0.0311 - accuracy: 0.992 - ETA: 10s - loss: 0.0294 - accuracy: 0.993 - ETA: 10s - loss: 0.0279 - accuracy: 0.993 - ETA: 10s - loss: 0.0266 - accuracy: 0.993 - ETA: 10s - 

1280/1325 [===========================>..] - ETA: 9s - loss: 0.0010 - accuracy: 1.00 - ETA: 9s - loss: 0.1471 - accuracy: 0.91 - ETA: 10s - loss: 0.0909 - accuracy: 0.950 - ETA: 10s - loss: 0.0649 - accuracy: 0.964 - ETA: 9s - loss: 0.0568 - accuracy: 0.972 - ETA: 9s - loss: 0.0466 - accuracy: 0.97 - ETA: 9s - loss: 0.0394 - accuracy: 0.98 - ETA: 9s - loss: 0.0380 - accuracy: 0.98 - ETA: 9s - loss: 0.0335 - accuracy: 0.98 - ETA: 9s - loss: 0.0555 - accuracy: 0.97 - ETA: 9s - loss: 0.0503 - accuracy: 0.97 - ETA: 9s - loss: 0.0472 - accuracy: 0.97 - ETA: 9s - loss: 0.0434 - accuracy: 0.98 - ETA: 9s - loss: 0.0402 - accuracy: 0.98 - ETA: 9s - loss: 0.0378 - accuracy: 0.98 - ETA: 9s - loss: 0.0467 - accuracy: 0.97 - ETA: 9s - loss: 0.0444 - accuracy: 0.97 - ETA: 9s - loss: 0.0419 - accuracy: 0.97 - ETA: 9s - loss: 0.0396 - accuracy: 0.97 - ETA: 9s - loss: 0.0377 - accuracy: 0.98 - ETA: 9s - loss: 0.0358 - accuracy: 0.98 - ETA: 8s - loss: 0.0342 - accuracy: 0.98 - ETA: 8s - loss: 0.0342 - a

1325/1325 [==============================] - ETA: 0s - loss: 0.0544 - accuracy: 0.98 - ETA: 0s - loss: 0.0543 - accuracy: 0.98 - ETA: 0s - loss: 0.0541 - accuracy: 0.98 - ETA: 0s - loss: 0.0540 - accuracy: 0.98 - ETA: 0s - loss: 0.0538 - accuracy: 0.98 - ETA: 0s - loss: 0.0537 - accuracy: 0.98 - ETA: 0s - loss: 0.0535 - accuracy: 0.98 - ETA: 0s - loss: 0.0535 - accuracy: 0.98 - ETA: 0s - loss: 0.0561 - accuracy: 0.98 - 15s 11ms/step - loss: 0.0561 - accuracy: 0.9842 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 5/50


...

1325/1325 [==============================] - ETA: 11s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 3.5763e-08 - accuracy: 1.000 - ETA: 12s - loss: 2.5545e-08 - accuracy: 1.000 - ETA: 12s - loss: 1.9868e-08 - accuracy: 1.000 - ETA: 12s - loss: 1.7881e-08 - accuracy: 1.000 - ETA: 12s - loss: 6.2863e-05 - accuracy: 1.000 - ETA: 12s - loss: 2.1687e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.8976e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.6868e-04 - accuracy: 1.000 - ETA: 12s - loss: 2.0700e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.8819e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.7250e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.5924e-04 - accuracy: 1.000 - ETA: 12s - loss: 1.4787e-04 - accuracy: 1.000 - ETA: 12s - loss: 0.0053 - accuracy: 1.0000    - ETA: 12s - loss: 0.0052 - accuracy: 1.000 - ETA: 12s - loss: 0.0049 - accuracy: 1.000 - ETA: 12s - loss: 0.0046 - accuracy: 1.000 - ETA: 12s - loss: 0.0043 - accuracy: 1.000 - ETA: 12s - 

1325/1325 [==============================] - ETA: 11s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 4.0831e-05 - accuracy: 1.000 - ETA: 12s - loss: 3.1758e-05 - accuracy: 1.000 - ETA: 12s - loss: 2.5984e-05 - accuracy: 1.000 - ETA: 12s - loss: 2.1986e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.9055e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.6813e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.5879e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.4291e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.3610e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.2992e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.2427e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.1480e-05 - accuracy: 1.000 - ETA: 12s - loss: 1.0630e-05 - accuracy: 1.000 - ETA: 12s - loss: 9.8970e-06 - accuracy: 1.000 - ETA: 12s - loss: 9.2613e-06 - accuracy: 1.000 - ETA: 12s - loss: 8.7036e-06 - accuracy: 1.000 - ETA: 12s - loss: 8.2063e-06 - accuracy: 1.0

1084/1325 [=======================>......] - ETA: 11s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 11s - loss: 2.9802e-08 - accuracy: 1.000 - ETA: 12s - loss: 4.0172e-06 - accuracy: 1.000 - ETA: 12s - loss: 2.8694e-06 - accuracy: 1.000 - ETA: 12s - loss: 2.2318e-06 - accuracy: 1.000 - ETA: 12s - loss: 2.4898e-06 - accuracy: 1.000 - ETA: 12s - loss: 2.1067e-06 - accuracy: 1.000 - ETA: 12s - loss: 1.8258e-06 - accuracy: 1.000 - ETA: 12s - loss: 2.7838e-06 - accuracy: 1.000 - ETA: 11s - loss: 2.4908e-06 - accuracy: 1.000 - ETA: 11s - loss: 2.0721e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.8919e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.7581e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.6279e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.5156e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.4178e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.3319e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.2558e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.1879e-05 - accuracy: 1.000 - ETA: 11s - loss: 1.1270e-05 - accuracy: 1.0

1325/1325 [==============================] - ETA: 2s - loss: 2.6116e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5926e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5738e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5554e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5372e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5193e-04 - accuracy: 1.00 - ETA: 2s - loss: 2.5016e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4842e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4670e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4500e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4333e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4168e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.4005e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.3845e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.3686e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.3530e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.3376e-04 - accuracy: 1.00 - ETA: 1s - loss: 2.3223e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.3073e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.2925e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.2778e-04 - accura

1325/1325 [==============================] - ETA: 11s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 12s - loss: 3.1590e-07 - accuracy: 1.000 - ETA: 12s - loss: 2.2565e-07 - accuracy: 1.000 - ETA: 12s - loss: 1.7550e-07 - accuracy: 1.000 - ETA: 12s - loss: 1.4359e-07 - accuracy: 1.000 - ETA: 12s - loss: 1.2150e-07 - accuracy: 1.000 - ETA: 12s - loss: 1.0530e-07 - accuracy: 1.000 - ETA: 12s - loss: 9.4666e-08 - accuracy: 1.000 - ETA: 11s - loss: 8.4701e-08 - accuracy: 1.000 - ETA: 11s - loss: 7.6634e-08 - accuracy: 1.000 - ETA: 11s - loss: 6.9970e-08 - accuracy: 1.000 - ETA: 11s - loss: 6.4373e-08 - accuracy: 1.000 - ETA: 11s - loss: 5.9604e-08 - accuracy: 1.000 - ETA: 11s - loss: 5.5494e-08 - accuracy: 1.000 - ETA: 11s - loss: 5.5759e-08 - accuracy: 1.000 - ETA: 11s - loss: 8.7786e-05 - accuracy: 1.000 - ETA: 11s - loss: 8.2770e-05 - accuracy: 1.000 - ETA: 11s - loss: 7.8296e-05 - accuracy: 1.000 - ETA: 11s - loss: 7.4281e-05 - accuracy: 1.0


### Evaluate trained model on the feature’s and label’s test sets.

In [29]:
model_evaluation_history = model.evaluate(X_test, y_test)

415/415 [==============================] - ETA:  - ETA:  - ETA:  - 0s 369us/step


In [30]:
### Save model in a folder 
RootFolder

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles'

In [31]:
# Saving Model
model_name = 'video combined_video-2vdo model_jj.h5'
model.save(RootFolder+'/'+model_name)

### How to load the model from disk

In [32]:
RootFolder

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles'

In [33]:
model_name = 'video combined_video-2vdo model_jj.h5'



In [34]:
from tensorflow import keras

model = keras.models.load_model(RootFolder+'/'+model_name)


#### Here

### Evaluating our Video Classification Model


In [35]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import pafy

In [36]:
# read testFrames csv
testFrames  = pd.read_csv(RootFolder+"/testFrames.csv")
testFrames.shape

(319, 3)

In [37]:
testFrames.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [38]:
# Extract each vdo from testFrames 
testSetVDOlist = testFrames["FrameFilename"].tolist()
print(testSetVDOlist)

['v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c

In [39]:
print(len(testSetVDOlist))

# Remove Duplicates From vdo list - as a vdo file has many frame images
testSetVDOlist = list(dict.fromkeys(testSetVDOlist))
print(len(testSetVDOlist))
 

319
76


In [40]:
testSetVDOlist

['v_Basketball_g01_c01.avi',
 'v_Basketball_g01_c02.avi',
 'v_Basketball_g01_c03.avi',
 'v_Basketball_g01_c04.avi',
 'v_Basketball_g01_c05.avi',
 'v_Basketball_g01_c06.avi',
 'v_Basketball_g01_c07.avi',
 'v_Basketball_g02_c01.avi',
 'v_Basketball_g02_c02.avi',
 'v_Basketball_g02_c03.avi',
 'v_Basketball_g02_c04.avi',
 'v_Basketball_g02_c05.avi',
 'v_Basketball_g02_c06.avi',
 'v_Basketball_g03_c01.avi',
 'v_Basketball_g03_c02.avi',
 'v_Basketball_g03_c03.avi',
 'v_Basketball_g03_c04.avi',
 'v_Basketball_g03_c05.avi',
 'v_Basketball_g03_c06.avi',
 'v_Basketball_g04_c01.avi',
 'v_Basketball_g04_c02.avi',
 'v_Basketball_g04_c03.avi',
 'v_Basketball_g04_c04.avi',
 'v_Basketball_g05_c01.avi',
 'v_Basketball_g05_c02.avi',
 'v_Basketball_g05_c03.avi',
 'v_Basketball_g05_c04.avi',
 'v_Basketball_g06_c01.avi',
 'v_Basketball_g06_c02.avi',
 'v_Basketball_g06_c03.avi',
 'v_Basketball_g06_c04.avi',
 'v_Basketball_g07_c01.avi',
 'v_Basketball_g07_c02.avi',
 'v_Basketball_g07_c03.avi',
 'v_Basketball

In [41]:
# separating the target
# To map the predicted categories with the actual categories, we will use train frame class:
y = trainFrames['class']
y = pd.get_dummies(y)

In [42]:
y

,Basketball,SoccerPenalty
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
2067,0,1
2068,0,1
2069,0,1
2070,0,1


### Example

In [43]:
testSetVDOlist[1]

'v_Basketball_g01_c02.avi'

In [44]:
temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[1])]
temp_df.head()

,FrameFilename,FullPathName,class
5,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
6,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
7,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
8,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
9,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [45]:
temp_df = temp_df.reset_index(drop=True)  # reset index
temp_df.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [46]:
print(temp_df['class'][0])

Basketball


In [47]:

    
for i in range(len(testSetVDOlist)):
    print(testSetVDOlist[i])
    temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[i])]
    temp_df = temp_df.reset_index(drop=True)  # reset index
    print(temp_df['class'][0])
    #print(temp_df.shape)

v_Basketball_g01_c01.avi
Basketball
v_Basketball_g01_c02.avi
Basketball
v_Basketball_g01_c03.avi
Basketball
v_Basketball_g01_c04.avi
Basketball
v_Basketball_g01_c05.avi
Basketball
v_Basketball_g01_c06.avi
Basketball
v_Basketball_g01_c07.avi
Basketball
v_Basketball_g02_c01.avi
Basketball
v_Basketball_g02_c02.avi
Basketball
v_Basketball_g02_c03.avi
Basketball
v_Basketball_g02_c04.avi
Basketball
v_Basketball_g02_c05.avi
Basketball
v_Basketball_g02_c06.avi
Basketball
v_Basketball_g03_c01.avi
Basketball
v_Basketball_g03_c02.avi
Basketball
v_Basketball_g03_c03.avi
Basketball
v_Basketball_g03_c04.avi
Basketball
v_Basketball_g03_c05.avi
Basketball
v_Basketball_g03_c06.avi
Basketball
v_Basketball_g04_c01.avi
Basketball
v_Basketball_g04_c02.avi
Basketball
v_Basketball_g04_c03.avi
Basketball
v_Basketball_g04_c04.avi
Basketball
v_Basketball_g05_c01.avi
Basketball
v_Basketball_g05_c02.avi
Basketball
v_Basketball_g05_c03.avi
Basketball
v_Basketball_g05_c04.avi
Basketball
v_Basketball_g06_c01.avi
Bas

In [48]:
#print(temp_df['class'][i])
temp_df.head()

,FrameFilename,FullPathName,class
0,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
1,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
2,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
3,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty


### end of Example

In [49]:
# for loop to read and store test frames
# creating two lists to store predicted and actual tags
predict = []
actual = []
indent_variable   = []

for i in range(len(testSetVDOlist)):
    #print(testSetVDOlist[i])
    temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[i])] # all frames of a particular vdo
    temp_df = temp_df.reset_index(drop=True)  # reset index
    #print(temp_df.shape)  
   
    prediction_images = []
    
    # appending the actual class of the video
    
    actual.append(temp_df['class'][0])
    #print("Actual:" , actual)

    # appending the vdo file name as indent
    indent_variable.append(temp_df['FrameFilename'][0])
    #print("Indent:" ,indent_variable)

    for i in tqdm(range(temp_df.shape[0])):  ### Loop through all frames of a particular vdo
        
 
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(temp_df['FullPathName'][i], target_size=(224,224,3))
        # converting it to array
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img/255
        # appending the image to the image list
        prediction_images.append(img)
        #print("Prediction:" , prediction_images)
        
    # converting the list to numpy array
    prediction_images = np.array(prediction_images)

    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)

    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)

    # predicting class for each array.
    # predict_classes() function on model in order to predict the class values for each instance in the array.
    prediction = model.predict_classes(prediction_images)
    #print("Prediction:" , prediction)
    # appending the mode of predictions (Class that occurs on highest frrequency) in predict list to assign the class to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 81.96it/s]


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 71.43it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 100.00it/s]


In [50]:
prediction

array([1, 1, 0, 0], dtype=int64)

In [51]:
y.columns.values[s.mode(prediction)[0][0]]

'Basketball'

In [52]:
# Calling DataFrame constructor after zipping
# both lists, with columns specified
EvalFinalOutput = pd.DataFrame(list(zip(indent_variable, actual ,predict )),
               columns =['VdOFileName', 'ActualClass' , 'PredictedClass'])
EvalFinalOutput

,VdOFileName,ActualClass,PredictedClass
0,v_Basketball_g01_c01.avi,Basketball,Basketball
1,v_Basketball_g01_c02.avi,Basketball,Basketball
2,v_Basketball_g01_c03.avi,Basketball,Basketball
3,v_Basketball_g01_c04.avi,Basketball,Basketball
4,v_Basketball_g01_c05.avi,Basketball,Basketball
...,...,...,...
71,v_SoccerPenalty_g07_c02.avi,SoccerPenalty,SoccerPenalty
72,v_SoccerPenalty_g07_c03.avi,SoccerPenalty,SoccerPenalty
73,v_SoccerPenalty_g07_c04.avi,SoccerPenalty,SoccerPenalty
74,v_SoccerPenalty_g07_c05.avi,SoccerPenalty,SoccerPenalty


In [53]:
EvalFinalOutput.to_csv(RootFolder+'/EvalFinalOutput.csv',header=True, index=False)